# My first graph in tensorflow

In [76]:
import tensorflow as tf

In [88]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def
import tensorflow as tf

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [78]:
import tensorflow as tf
>>> hello = tf.constant('Hello, TensorFlow!')
>>> sess = tf.Session()
>>> print(sess.run(hello))

b'Hello, TensorFlow!'


In [4]:
print('test of the new tensorflow')

test of the new tensorflow


In [5]:
x =tf.Variable(3, name = 'x')
y =tf.Variable(3, name = 'y')
f = x*x*y + y + 2

In [6]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    f.eval()

In [7]:
graph = tf.Graph()

In [8]:
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph


True

In [79]:
show_graph(graph)

In [10]:
x2.graph is graph

True

In [11]:
tf.reset_default_graph()

In [20]:
import numpy as np
import sklearn as sk
from sklearn.datasets import fetch_california_housing

In [38]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [49]:
import pandas as pd
pd.DataFrame(housing.target.reshape(-1,1)).head()

,0,1,2,3,4,5,6,7,8,9,...,20630,20631,20632,20633,20634,20635,20636,20637,20638,20639
0,4.526,3.585,3.521,3.413,3.422,2.697,2.992,2.414,2.267,2.611,...,1.12,1.072,1.156,0.983,1.168,0.781,0.771,0.923,0.847,0.894


In [50]:
pd.DataFrame(np.c_[np.ones((m,1)),housing.data]).head()

,0,1,2,3,4,5,6,7,8
0,1.0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,1.0,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,1.0,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,1.0,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,1.0,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [60]:
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    result = theta.eval()

print(result)

[[-3.63715744e+01]
 [ 4.37795252e-01]
 [ 9.53791849e-03]
 [-1.08239025e-01]
 [ 6.47155583e-01]
 [-3.69874488e-06]
 [-3.80105944e-03]
 [-4.15575713e-01]
 [-4.28025067e-01]]


In [63]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

In [93]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

graphLR = tf.Graph()

with graphLR.as_default():
    X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
    y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
    theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name ='theta')
    y_pred = tf.matmul(X, theta, name="predictions")
    error = y_pred - y
    mse = tf.losses.mean_squared_error(y,y_pred)
    gradients = 2/m * tf.matmul(tf.transpose(X),error)
    training_op = tf.assign(theta, theta-learning_rate*gradients)

    
        

In [94]:
show_graph(graphLR)

In [81]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
        init.run()

        for epoch in range(n_epochs):
            if epoch % 100 == 0:
                print("Epoch ", epoch, "MSE =", mse.eval())
            sess.run(training_op)

    

ValueError: Cannot use the default session to execute operation: the operation's graph is different from the session's graph. Pass an explicit session to run(session=sess).

In [66]:
n

8